# 7. LangSmith を使った RAG アプリケーションの評価


In [1]:
!pip install numpy==1.26.4

In [3]:
# 【注意】
# 上記の `!pip install numpy==1.26.4` を実行したあと、
# Google Colab 上部のメニューから「ランタイム」の「セッションを再起動する」を実行してください。
# その後このセルを実行して `1.26.4` と表示されることを確認してください。

import numpy as np

print(np.__version__)
assert np.__version__ == "1.26.4"

1.26.4


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 7.4. Ragas による合成テストデータの生成


### パッケージのインストール


In [1]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.10.6

# 📦 すべてのLangChain関連ライブラリをまとめて最新の安定版にアップデートするよ！
#!pip install -U langchain langchain-core langchain-openai langchain-community langsmith

### 検索対象のドキュメントのロード


In [5]:
from langchain_community.document_loaders import GitLoader  # GitHubからファイルを取り出す道具を使うよ

def file_filter(file_path: str) -> bool:  # ファイルの名前を見て、使うかどうかを決めるルールを作るよ
    return file_path.endswith(".mdx")  # 「.mdx」で終わるファイルだけOKってルールだよ

loader = GitLoader(  # GitHubからファイルを読み込む準備をするよ
    clone_url="https://github.com/langchain-ai/langchain",  # GitHubのページの場所（URL）だよ
    repo_path="./langchain",  # パソコンの中に保存するときのフォルダの名前だよ
    branch="master",  # GitHubの中の「master」っていうメインのページを見るよ
    file_filter=file_filter,  # さっきの「.mdxだけOK」のルールを使うよ
)

documents = loader.load()  # ファイルを実際に読み込んで取り出すよ
print(len(documents))  # 何個のファイルを読み込んだか数えて表示するよ


418


In [4]:
from langchain_community.document_loaders import GitLoader  # GitHubからファイルを取り出す道具を使うよ

def file_filter(file_path: str) -> bool:  # ファイルの名前を見て、使うかどうかを決めるルールを作るよ
    return file_path.endswith(".mdx")  # 「.mdx」で終わるファイルだけOKってルールだよ

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)

documents = loader.load()  # ファイルを実際に読み込んで取り出すよ
print(len(documents))  # 何個のファイルを読み込んだか数えて表示するよ


280


### Ragas による合成テストデータ生成の実装


In [5]:
for document in documents:  # すべての読み込んだ文書を順番に見るよ
    document.metadata["filename"] = document.metadata["source"]  # 元の場所（source）をファイル名としてメモしておくよ


#### 【注意】既知のエラーについて

以下のコードで gpt-4o を使用すると OpenAI API の Usage tier 次第で RateLimitError が発生することが報告されています。

OpenAI API の Usage tier については公式ドキュメントの以下のページを参照してください。

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

このエラーが発生した場合は、以下のどちらかの対応を実施してください。

1. 同じ Tier でも gpt-4o よりレートリミットの高い gpt-4o-mini を使用する
   - この場合、生成される合成テストデータの品質は低くなることが想定されます
2. 課金などにより Tier を上げる
   - Tier 2 で RateLimitError が発生しないことを確認済みです (2024 年 10 月 31 日時点)

##### 2025/3/15 追記

LangChain のドキュメントの増加により、gpt-4o-mini を使用しても Tier 1 ではエラーが発生することが報告されています。

その場合、上部のコードの GitHub からドキュメントをロードする箇所で、以下のように `langchain==0.2.13` という動作確認済みのバージョンを指定するようにしてください。

```python
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="langchain==0.2.13",
    file_filter=file_filter,
)
```


In [8]:
import nest_asyncio  # ノートブックで「待ってね処理（非同期）」を使いやすくするための道具だよ
from ragas.testset.generator import TestsetGenerator  # テスト問題を作ってくれるクラスを読み込むよ
from ragas.testset.evolutions import simple, reasoning, multi_context  # 問題の種類（かんたん・考える・いくつかの情報を使う）を使うよ
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # OpenAIのチャットと文章の特徴づけ（埋め込み）を使うよ

nest_asyncio.apply()  # ノートブックの中でも「非同期」がうまく動くようにするよ（おまじないみたいなもの）

generator = TestsetGenerator.from_langchain(  # テスト作成ロボットを作るよ（文章を読んで質問を作る係と、評価する係を入れるよ）
    generator_llm=ChatOpenAI(model="gpt-4o"),  # 質問を作るためのAIを指定するよ（GPT-4o）
    critic_llm=ChatOpenAI(model="gpt-4o"),  # 質問がよいかチェックするAIもGPT-4oを使うよ
    embeddings=OpenAIEmbeddings(),  # 文章を特徴にする道具を使うよ
)

testset = generator.generate_with_langchain_docs(  # 文章からテスト問題を作るよ
    documents,  # 入力する文章のリストだよ
    test_size=4,  # 問題を4問作るよ
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},  # かんたん問題50%、考える問題25%、いくつかの文を使う問題25%で出題するよ
    #raise_exceptions=False  # ←これで原因をもっと見つけやすくなるよ
)


embedding nodes:   0%|          | 0/902 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-or4ttQfngACg3ZTwiOEu2LBV on tokens per min (TPM): Limit 30000, Used 29196, Requested 1314. Please try again in 1.02s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [9]:
import nest_asyncio  # ノートブックで「待ってね処理（非同期）」を使いやすくするための道具だよ
from ragas.testset.generator import TestsetGenerator  # テスト問題を作ってくれるクラスを読み込むよ
from ragas.testset.evolutions import simple, reasoning, multi_context  # 問題の種類（かんたん・考える・いくつかの情報を使う）を使うよ
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # OpenAIのチャットと文章の特徴づけ（埋め込み）を使うよ

nest_asyncio.apply()  # ノートブックの中でも「非同期」がうまく動くようにするよ（おまじないみたいなもの）

generator = TestsetGenerator.from_langchain(  # テスト作成ロボットを作るよ（文章を読んで質問を作る係と、評価する係を入れるよ）
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),  # 質問を作るためのAIを指定するよ（GPT-4o）
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),  # 質問がよいかチェックするAIもGPT-4oを使うよ
    embeddings=OpenAIEmbeddings(),  # 文章を特徴にする道具を使うよ
)

testset = generator.generate_with_langchain_docs(  # 文章からテスト問題を作るよ
    documents,  # 入力する文章のリストだよ
    test_size=4,  # 問題を4問作るよ
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},  # かんたん問題50%、考える問題25%、いくつかの文を使う問題25%で出題するよ
    #raise_exceptions=False  # ←これで原因をもっと見つけやすくなるよ
)


embedding nodes:   0%|          | 0/902 [00:00<?, ?it/s]

Generating:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
testset.to_pandas()  # 作ったテストを表（データフレーム）に変えて見やすくするよ

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of BibTeX in academic and ...,[# BibTeX\n\n>[BibTeX](https://www.ctan.org/pk...,BibTeX is a file format and reference manageme...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What is the purpose of the Document Loader in ...,[# Telegram\n\n>[Telegram Messenger](https://w...,The answer to given question is not present in...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,What role does PromptLayer play in prompt engi...,[# PromptLayer\n\n>[PromptLayer](https://docs....,PromptLayer is a platform for prompt engineeri...,reasoning,[{'source': 'docs/docs/integrations/providers/...,True
3,How does the OpenAI API key function as an env...,[# MLflow Deployments for LLMs\n\n>[The MLflow...,The OpenAI API key functions as an environment...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


### LangSmith の Dataset の作成


In [12]:
from langsmith import Client  # LangSmithというサービスを使うための道具を呼び出すよ

dataset_name = "agent-book"  # 保存するノート（データセット）の名前を決めるよ

client = Client()  # LangSmithとやり取りする係を用意するよ

# もしすでに同じ名前のノートがあったら削除するよ（新しく作り直すため）
if client.has_dataset(dataset_name=dataset_name):  # 同じ名前のノートがあるかチェック
    client.delete_dataset(dataset_name=dataset_name)  # あれば消すよ

# 新しいノートを作るよ
dataset = client.create_dataset(dataset_name=dataset_name)

### 合成テストデータの保存


In [13]:
# 入力（質問）・出力（答え）・メモ（どこから来たかなど）を入れる箱を用意するよ
inputs = []  # 質問だけを入れるリスト
outputs = []  # 答え（正解と文脈）を入れるリスト
metadatas = []  # どんな進化（問題タイプ）か、どの資料から来たかを入れるリスト

# 作ったテストデータを1つずつ処理して、3つのリストに分けていくよ
for testset_record in testset.test_data:  # テストデータを順番に見ていくよ
    inputs.append(  # 質問の情報だけを取り出して追加するよ
        {
            "question": testset_record.question,  # テストの質問を入れるよ
        }
    )
    outputs.append(  # 答えとその元になった文を追加するよ
        {
            "contexts": testset_record.contexts,  # 文脈（どんな文章を見て答えるか）
            "ground_truth": testset_record.ground_truth,  # 正しい答え（正解）だよ
        }
    )
    metadatas.append(  # どのファイルから来たか、問題のタイプは何かも入れておくよ
        {
            "source": testset_record.metadata[0]["source"],  # 元のファイルの名前
            "evolution_type": testset_record.evolution_type,  # 問題のタイプ（かんたん、考える、など）
        }
    )

In [14]:
# 3つのリストをLangSmithのノートにまとめて保存するよ
client.create_examples(
    inputs=inputs,  # 質問のリスト
    outputs=outputs,  # 答えのリスト
    metadata=metadatas,  # メモ（情報）のリスト
    dataset_id=dataset.id,  # どのノートに書き込むかを指定するよ
)

## 7.5. LangSmith と Ragas を使ったオフライン評価の実装


### カスタム Evaluator の実装


In [15]:
from typing import Any  # なんでも入る箱（型）を使う準備だよ

# AIのしくみに必要な道具たちを呼び出すよ
from langchain_core.embeddings import Embeddings  # 文章の特徴を数で表す道具
from langchain_core.language_models import BaseChatModel  # チャット型AIのベースの道具
from langsmith.schemas import Example, Run  # LangSmithのテストデータや実行の記録を使うよ

# Ragasっていう評価ツールのラッパー（包む道具）を呼ぶよ
from ragas.embeddings import LangchainEmbeddingsWrapper  # 埋め込みを包む道具
from ragas.llms import LangchainLLMWrapper  # LLM（AI）を包む道具
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM  # 評価のルールを決める道具

# 採点マシンを作るクラスだよ
class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric  # どんなルールで評価するかを保存するよ

        # AIを使うルールの場合は、AIを設定するよ
        if isinstance(self.metric, MetricWithLLM):  # もしAIが必要なルールなら
            self.metric.llm = LangchainLLMWrapper(llm)  # AIをラップして設定するよ

        # 埋め込みが必要なルールの場合は、設定するよ
        if isinstance(self.metric, MetricWithEmbeddings):  # もし埋め込みが必要なルールなら
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)  # 包んで設定するよ

    # 実行したAIの答え（run）と正解の例（example）を使って点数を出すよ
    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]  # 文脈を文字として取り出すよ

        # Ragasのscore関数で点数を計算するよ
        score = self.metric.score({
            "question": example.inputs["question"],  # 質問
            "answer": run.outputs["answer"],  # AIの答え
            "contexts": context_strs,  # 文脈（ヒント）
            "ground_truth": example.outputs["ground_truth"],  # 本当の正解
        })

        return {"key": self.metric.name, "score": score}  # 結果を返すよ（何の評価かと点数）


In [16]:
# --- 評価に使うAIモデルやルールの設定 ---

from langchain_openai import ChatOpenAI, OpenAIEmbeddings  # OpenAIのAIモデルや埋め込みモデルを呼ぶよ
from ragas.metrics import answer_relevancy, context_precision  # 評価ルールを2つ使うよ

metrics = [context_precision, answer_relevancy]  # 評価ルールをリストにまとめるよ

llm = ChatOpenAI(model="gpt-4o", temperature=0)  # AIモデルを冷静モードで用意するよ
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 埋め込みモデルも用意するよ

# 各評価ルールに対して採点マシンを作るよ
evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate  # 評価マシンを1つずつ作るよ
    for metric in metrics  # ルールの数だけ繰り返すよ
]

### 推論の関数の実装


In [17]:
from langchain_chroma import Chroma  # Chromaっていう検索用の箱（ベクトルDB）を使うよ
from langchain_openai import OpenAIEmbeddings  # 文章を特徴ベクトルに変える道具（OpenAI埋め込み）を使うよ

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 小さいモデルで文章を特徴に変える道具を作るよ
db = Chroma.from_documents(documents, embeddings)  # 読み込んだ文書をChromaに入れて、検索できるようにするよ


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [18]:
from langchain_core.output_parsers import StrOutputParser  # AIの返事をただの文章に整える道具を使うよ
from langchain_core.prompts import ChatPromptTemplate  # AIへの質問のひな形（テンプレート）を作る道具だよ
from langchain_core.runnables import RunnableParallel, RunnablePassthrough  # 処理をつなげたり並列で動かす道具だよ
from langchain_openai import ChatOpenAI  # OpenAIのチャット型AIを使うよ

# 質問テンプレートを作るよ（文脈に基づいて質問に答えてね、という内容）
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # 落ち着いた（ぶれない）性格のAIを使うよ

retriever = db.as_retriever()  # Chromaを検索ロボットに変身させるよ

# 質問と文脈を並列で準備して、AIに答えさせるチェーンを作るよ
chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),  # 入力された質問をそのまま渡すよ
        "context": retriever,  # 質問に関連する文書を探すよ
    }
).assign(answer=prompt | model | StrOutputParser())  # AIに答えさせて、キレイな文章に整えて「answer」に入れるよ


In [19]:
# AIに質問して、答えと使った文脈を返す関数を作るよ
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]  # 入ってきた質問を取り出すよ
    output = chain.invoke(question)  # AIに質問して答えてもらうよ
    return {
        "contexts": output["context"],  # AIが使った文脈（参考にした情報）を返すよ
        "answer": output["answer"],  # AIの答えを返すよ
    }


### オフライン評価の実装・実行


In [20]:
from langsmith.evaluation import evaluate  # LangSmithで点数をつける道具を読み込むよ

# AIの答えを評価して、どれだけ上手にできたかをスコアにするよ
evaluate(
    predict,  # さっき作ったAIへの質問→回答の関数だよ
    data="agent-book",  # 評価に使うテストデータセットの名前だよ
    evaluators=evaluators,  # どんなルールで採点するかを渡すよ
)

View the evaluation results for experiment: 'mealy-manager-4' at:
https://smith.langchain.com/o/102cbe0e-259f-4adf-b526-d4a02b4b3bc4/datasets/0e5ea40e-7639-4260-b362-b8b4b3fed05b/compare?selectedSessions=b90a9eba-a48b-4f0a-84fb-31ad2d0f9d73




0it [00:00, ?it/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.context_precision,feedback.answer_relevancy,execution_time,example_id,id
0,What is the purpose of BibTeX in academic and ...,[page_content='# BibTeX\n\n>[BibTeX](https://w...,BibTeX is a file format and reference manageme...,None,[# BibTeX\n\n>[BibTeX](https://www.ctan.org/pk...,BibTeX is a file format and reference manageme...,1.0,0.849839,1.662847,4bf59030-40cc-4bc6-aee2-f3a5acd4c335,b04358a9-f05f-4db9-820f-f095aeeeb189
1,What is the purpose of the Document Loader in ...,[page_content='# Telegram\n\n>[Telegram Messen...,The purpose of the Document Loader in the cont...,None,[# Telegram\n\n>[Telegram Messenger](https://w...,The answer to given question is not present in...,0.0,1.000000,2.523482,04a81ac4-45d1-436a-bd28-9de015aafa50,50d43f7f-fbd8-4b91-9e4b-045b3d1dc45a
2,What role does PromptLayer play in prompt engi...,[page_content='# PromptLayer\n\n>[PromptLayer]...,PromptLayer is a platform designed for prompt ...,None,[# PromptLayer\n\n>[PromptLayer](https://docs....,PromptLayer is a platform for prompt engineeri...,1.0,0.747489,2.071526,8e307df4-6da6-454e-b413-42ce12d23ff8,fa256397-5587-4cd5-94c0-0828096b982c
3,How does the OpenAI API key function as an env...,[page_content='# MLflow AI Gateway\n\n:::warni...,In both MLflow Deployments and the deprecated ...,None,[# MLflow Deployments for LLMs\n\n>[The MLflow...,The OpenAI API key functions as an environment...,1.0,0.888456,7.698351,41bd499a-e697-46c9-a153-2e9490f38b93,ab03338d-8850-45a2-b82e-a4c732bbe095


## LangSmith を使ったオンライン評価の実装


### フィードバックボタンを表示する関数の実装


In [21]:
from uuid import UUID  # ランダムなID（名前のかわり）を扱うための道具だよ

import ipywidgets as widgets  # ボタンやUIを作る道具だよ
from IPython.display import display  # Jupyterでボタンなどを表示するための道具だよ
from langsmith import Client  # LangSmithにフィードバックを送るための道具だよ

# フィードバックのボタンを表示する関数をつくるよ
def display_feedback_buttons(run_id: UUID) -> None:
    # Goodボタンを作るよ（緑の親指マーク👍）
    good_button = widgets.Button(
        description="Good",  # ボタンに書かれる文字
        button_style="success",  # 緑色のボタン
        icon="thumbs-up",  # 親指マーク
    )
    # Badボタンを作るよ（赤の親指下げマーク👎）
    bad_button = widgets.Button(
        description="Bad",  # ボタンに書かれる文字
        button_style="danger",  # 赤色のボタン
        icon="thumbs-down",  # 親指下げマーク
    )

    # ボタンが押されたときの動きを決める関数だよ
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:  # Goodが押されたとき
            score = 1  # 点数は1（いいね！）
        elif button == bad_button:  # Badが押されたとき
            score = 0  # 点数は0（うーん…）
        else:
            raise ValueError(f"Unknown button: {button}")  # よくわからないボタンが来たらエラーにするよ

        client = Client()  # LangSmithとつながる準備をするよ
        client.create_feedback(run_id=run_id, key="thumbs", score=score)  # フィードバック（GoodかBad）を送るよ
        print("フィードバックを送信しました")  # 送ったよ！と表示するよ

    # ボタンが押されたら on_button_clicked を実行するように設定するよ
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # 画面にボタンを表示するよ
    display(good_button, bad_button)


### フィードバックボタンを表示


In [23]:
from langchain_core.tracers.context import collect_runs  # LangSmithで記録（トレース）するための道具だよ

# AIが答えたときの記録を集めるよ（どんな質問にどう答えたか記録されるよ）
with collect_runs() as runs_cb:
    output = chain.invoke("LangChainの概要を教えて")  # 「LangChainって何？」とAIに聞いてみるよ
    print(output["answer"])  # AIの答えを表示するよ
    run_id = runs_cb.traced_runs[0].id  # 一番最初の記録（Run）のIDを取り出すよ

# さっきの記録に対して、Good/Badボタンでフィードバックできるようにするよ
display_feedback_buttons(run_id)

LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。

- **開発**: LangChainのオープンソースのビルディングブロックやコンポーネント、サードパーティの統合を使用してアプリケーションを構築します。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。
- **生産化**: LangSmithを使用して、チェーンを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。
- **デプロイ**: LangGraphアプリケーションを本番環境向けのAPIやアシスタントに変換します。

LangChainは、`langchain-core`、`langchain-community`、`langchain`、`LangGraph`、`LangServe`、`LangSmith`などのオープンソースライブラリで構成されており、これらを組み合わせることで、強力で柔軟なアプリケーションを構築できます。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

フィードバックを送信しました
